In [1]:
import numpy as np
from scipy.stats import norm
from dataclasses import dataclass
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import pandas as pd

# SABR model
$$ dF_t = \sigma F^\beta dW_t $$
$$  d\sigma = \nu \sigma dZ_t $$
$$ E[dW_t dZ_t] = \rho $$
$$  \sigma(0) = \alpha $$


# Put-Call parity for forwards
$$ C_0 - P_0 = e^{-rT}(F_0 - K) $$

# Second order finite differences approximation for risk-neutral density
$$
f(K,T) \approx e^{rT} \,
\frac{
    C(K + \Delta K, T)
    - 2 C(K, T)
    + C(K - \Delta K, T)
}{
    (\Delta K)^2
}
$$

In [2]:
# Blacks formula for call option on forward
def Blacks_European_Call(F0, K, sigma, r, T):
    vol_sqrt_T = sigma * np.sqrt(T)
    d1 = (np.log(F0 / K) + 0.5 * sigma**2 * T) / vol_sqrt_T
    d2 = d1 - vol_sqrt_T

    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)

    return np.exp(-r * T) * (F0 * Nd1 - K * Nd2)


In [3]:
@dataclass
class SABR_parameters:
    beta: float
    nu: float
    rho: float
    alpha : float

In [4]:


# Hagans asymptotic solution and solution at F0 = K
def SABR_implied_sigma(F0, K, T, p:SABR_parameters):
    # ATM expansion 
    eps = 1e-07
    if abs(F0 - K) < eps:
        
        f_beta = F0**(1.0 - p.beta)
        sigma_0 = p.alpha / f_beta
        
       
        term1 = ((1.0 - p.beta)**2 / 24.0) * (p.alpha**2) / (F0**(2.0 * (1.0 - p.beta)))
        term2 = 0.25 * p.rho * p.beta * p.nu * p.alpha / (F0**(1.0 - p.beta))
        term3 = (2.0 - 3.0 * p.rho**2) * p.nu**2 / 24.0
        
        sigma_atm = sigma_0 * (1.0 + (term1 + term2 + term3) * T)
        return sigma_atm

    # General (non-ATM) case
    FK = F0 * K
    one_minus_beta = 1.0 - p.beta
   
    FK_beta = FK**(0.5 * one_minus_beta)

    log_fk = np.log(F0 / K)

  
    z = (p.nu / p.alpha) * FK_beta * log_fk
   
    if abs(z) < eps:
        x_z = 1.0 - 0.5 * p.rho * z  
    else:
        x_z = np.log((np.sqrt(1.0 - 2.0 * p.rho * z + z * z) + z - p.rho) / (1.0 - p.rho))

    chi_z = z / x_z

    # Prefactor A(F,K)
    A = p.alpha / FK_beta

    # I1 and I2 correction terms
    FK_pow_1mb = FK**one_minus_beta      
    FK_pow_2mb = FK**(2.0 * one_minus_beta)  

    I1 = ((one_minus_beta**2) / 24.0) * (p.alpha**2) / FK_pow_1mb \
         + 0.25 * p.rho * p.beta * p.nu * p.alpha / FK_beta \
         + (2.0 - 3.0 * p.rho**2) * (p.nu**2) / 24.0

    I2 = ((one_minus_beta**2) / 24.0) * (log_fk**2) / FK_pow_1mb \
         + ((one_minus_beta**4) / 1920.0) * (log_fk**4) / FK_pow_2mb

    sigma_black = A * chi_z * (1.0 + I1 * T + I2)
    return sigma_black

In [5]:
def SABR_European_call_forward(F0, K, T, r, p:SABR_parameters):
    sigma_black = SABR_implied_sigma(F0, K, T, p)
    return Blacks_European_Call(F0, K, sigma_black, r, T)

def SABR_European_put_forward(F0, K, T, r, p:SABR_parameters):
    call = SABR_European_call_forward(F0, K, T, r, p)
    return call - np.exp(-r * T) * (F0 - K)

In [6]:
def plot_Blacks_implied_vol_SABR(F0, T, r, beta, nu, rho, alpha):
    K_min, K_max = 0.2 * F0, 3 * F0
    K_grid = np.linspace(K_min, K_max, 500)
    dK = K_grid[1] - K_grid[0]
    hp = SABR_parameters(beta = beta, nu = nu, rho = rho, alpha = alpha)
    SABR_sigma = [SABR_implied_sigma(F0, strike, T, hp) * 100 for strike in K_grid]
    SABR_calls = [SABR_European_call_forward(F0, k, T, r, hp) for k in K_grid]
    prices = pd.DataFrame([K_grid, SABR_calls]).transpose()
    prices.columns = ["strike", "price"]
    prices["risk_neutral_density"] = np.exp(r*T) * ((-2 * prices["price"] + prices["price"].shift(1) + prices["price"].shift(-1)) / (dK)**2)
    fig, ax = plt.subplots(1, 2, figsize =  (12, 6))
    ax[0].plot(K_grid, SABR_sigma, color = 'black')
    ax[0].set_xlabel("Strike")
    ax[0].set_ylabel("Black's model implied volatility (%)")
    ax[0].set_title("Black's model implied volatility (%) against strike for SABR model")
    ax[0].grid(True)
    ax[1].plot(prices["strike"], prices["risk_neutral_density"], color = 'black')
    ax[1].set_xlabel("Forward Price $(F_T)$")
    ax[1].set_ylabel("$f(F_T)$")
    ax[1].set_title("Risk neutral density for SABR model")
    ax[1].grid(True)
    plt.show()

interact(plot_Blacks_implied_vol_SABR,
        F0=FloatSlider(value=40, min=1,  max=100, step=1,   description="F0"),
         T=FloatSlider(value=1.0, min=0.05, max=5.0, step=0.05, description="T"),
         r=FloatSlider(value=0.02, min=0.0, max=0.1, step=0.002, description="r"),
         q=FloatSlider(value=0.0, min=0.0, max=0.1, step=0.002, description="q"),
         beta=FloatSlider(value=0.01, min=0.01, max=1.00, step=0.01, description="beta"),
         nu=FloatSlider(value=0.12, min=0.001, max=0.5, step=0.005, description="nu"),
         rho=FloatSlider(value=0.00, min=-1.00, max=1.00, step=0.005, description="rho"),
         alpha=FloatSlider(value=1.0, min=0.01, max=4.00, step=0.005, description="alpha")
         )

interactive(children=(FloatSlider(value=40.0, description='F0', min=1.0, step=1.0), FloatSlider(value=1.0, des…

<function __main__.plot_Blacks_implied_vol_SABR(F0, T, r, beta, nu, rho, alpha)>